# Clase Práctica 1: Búsqueda
---

# Problem and Node

In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
import copy
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

# Queues

In [28]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

# Search Algorithms

Con best-first search al variar la función *f(n)* se tienen diferentes algoritmos de búsqueda.

In [29]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)



In [30]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
        
        
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result:
                return result
        return failure

# Bidirectional Best-First Search

In [90]:
def bidirectional_best_first_search(problem_f, f_f, problem_b, f_b, terminated):
    node_f = Node(problem_f.initial)
    node_b = Node(problem_f.goal)
    frontier_f, reached_f = PriorityQueue([node_f], key=f_f), {node_f.state: node_f}
    frontier_b, reached_b = PriorityQueue([node_b], key=f_b), {node_b.state: node_b}
    solution = failure
    while frontier_f and frontier_b and not terminated(solution, frontier_f, frontier_b):
        def S1(node, f):
            return str(int(f(node))) + ' ' + str(path_states(node))
        if f_f(frontier_f.top()) < f_b(frontier_b.top()):
            solution = proceed('f', problem_f, frontier_f, reached_f, reached_b, solution)
        else:
            solution = proceed('b', problem_b, frontier_b, reached_b, reached_f, solution)
    return solution

def inverse_problem(problem):
    if isinstance(problem, CountCalls):
        return CountCalls(inverse_problem(problem._object))
    else:
        inv = copy.copy(problem)
        inv.initial = inv.goal
        inv.goal = inv.initial
        inv.h = inv.reverse_h
        return inv

In [91]:
def bidirectional_uniform_cost_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        n_f, n_b = frontier_f.top(), frontier_b.top()
        return g(n_f) + g(n_b) > g(solution)
    return bidirectional_best_first_search(problem_f, g, inverse_problem(problem_f), g, terminated)

def bidirectional_astar_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        nf, nb = frontier_f.top(), frontier_b.top()
        return g(nf) + g(nb) > g(solution)
    problem_b = inverse_problem(problem_f)
    return bidirectional_best_first_search(problem_f, lambda n: g(n) + problem_f.h(n),
                                           problem_b, lambda n: g(n) + problem_b.h(n), 
                                           terminated)
   

def proceed(direction, problem, frontier, reached, reached2, solution):
    node = frontier.pop()
    for child in expand(problem, node):
        s = child.state
        if s not in reached or child.path_cost < reached[s].path_cost:
            frontier.add(child)
            reached[s] = child
            if s in reached2: # Frontiers collide; solution found
                solution2 = (join_nodes(child, reached2[s]) if direction == 'f' else
                             join_nodes(reached2[s], child))
                #print('solution', path_states(solution2), solution2.path_cost, 
                # path_states(child), path_states(reached2[s]))
                if solution2.path_cost < solution.path_cost:
                    solution = solution2
    return solution

S = path_states

#A-S-R + B-P-R => A-S-R-P + B-P
def join_nodes(nf, nb):
    """Join the reverse of the backward node nb to the forward node nf."""
    #print('join', S(nf), S(nb))
    join = nf
    while nb.parent is not None:
        cost = join.path_cost + nb.path_cost - nb.parent.path_cost
        join = Node(nb.parent.state, join, nb.action, cost)
        nb = nb.parent
        #print('  now join', S(join), 'with nb', S(nb), 'parent', S(nb.parent))
    return join
    
   

In [92]:
def swap(L, i, j):
    temp = L[i]
    L[i] = L[j]
    L[j] = temp

# Pancake Sorting Problems

Dada una pila de pancakes de varios tamaños, ¿puedes ordenarlos en una pila de tamaños decrecientes, del más grande en la parte inferior al más pequeño en la parte superior? Tienes una espátula con la que puedes voltear los `i` pancakes de arriba. Esto se muestra a continuación para `i = 3`; en la parte superior, la espátula agarra los primeros tres pancakes; en la parte inferior los vemos volteados:

![](images/Pancake_sort_operation.png)

¿Cuántas vueltas se necesitarán para ordenar toda la pila? Este es un problema interesante sobre el que Bill Gates ha [escrito](https://people.eecs.berkeley.edu/~christos/papers/Bounds%20For%20Sorting%20By%20Prefix%20Reversal.pdf). Una heurística razonable para este problema es la *gap heuristic*: si observamos panqueques vecinos, por ejemplo, el segundo más pequeño está al lado del tercero más pequeño, está bien; deben permanecer uno al lado del otro. Pero si el 2º más pequeño está al lado del 4º más pequeño, eso es malo: necesitaremos al menos un movimiento para separarlos e insertar el 3º más pequeño entre ellos. La heurística cuenta el número de vecinos que tienen una brecha como esta. En nuestra especificación del problema, los panqueques se clasifican por tamaño: el más pequeño es "1", el segundo más pequeño es "2", y así sucesivamente, y la representación de un estado es una tupla de estas clasificaciones, de arriba hacia abajo. Por lo tanto, el estado objetivo siempre es `(1, 2, ..., `*n*`)` y el estado inicial (superior) en el diagrama anterior es `(2, 1, 4, 6, 3, 5)` .


In [101]:
class PancakeProblem(Problem):
    """A PancakeProblem the goal is always `tuple(range(1, n+1))`, where the
    initial state is a permutation of `range(1, n+1)`. An act is the index `i` 
    of the top `i` pancakes that will be flipped."""
    
    def __init__(self, initial): 
        self.initial, self.goal = tuple(initial), tuple(sorted(initial))
        self.pos_in_initial = {}
        for i in range(0, len(initial)):
            self.pos_in_initial[self.initial[i]] = i
    
    def actions(self, state):
        return list(range(len(state)))

    def result(self, state, flip_index):
        answer = list(state)
        for i in range(0, (flip_index>>1) + 1):
            swap(answer, flip_index - i, i)
        return tuple(answer)
    
    def h(self, node):
        gaps = 0
        for i in range(0, len(node.state) - 1):
            if abs(node.state[i] - node.state[i + 1]) > 1:
                gaps = gaps + 1 
        return gaps
    
    def reverse_h(self, node):
        gaps = 0
        for i in range(0, len(node.state) - 1):
            if abs(self.pos_in_initial[node.state[i]] - self.pos_in_initial[node.state[i + 1]]) > 1:
                gaps = gaps + 1 
        return gaps

SyntaxError: invalid syntax (1973707439.py, line 31)

In [99]:
c0 = PancakeProblem((2, 1, 4, 6, 3, 5))
c1 = PancakeProblem((4, 6, 2, 5, 1, 3))
c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4))
c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4))
c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8))

In [100]:
# Solve a pancake problem
sln = bidirectional_astar_search(c4)
path_actions(sln), path_states(bidirectional_astar_search(c4))

TypeError: 'dict' object is not callable

In [83]:
sln = astar_search(c4)
path_actions(sln), path_states(astar_search(c4))

([4, 8, 7, 6, 5, 4, 3, 2, 1],
 [(1, 3, 5, 7, 9, 2, 4, 6, 8),
  (9, 7, 5, 3, 1, 2, 4, 6, 8),
  (8, 6, 4, 2, 1, 3, 5, 7, 9),
  (7, 5, 3, 1, 2, 4, 6, 8, 9),
  (6, 4, 2, 1, 3, 5, 7, 8, 9),
  (5, 3, 1, 2, 4, 6, 7, 8, 9),
  (4, 2, 1, 3, 5, 6, 7, 8, 9),
  (3, 1, 2, 4, 5, 6, 7, 8, 9),
  (2, 1, 3, 4, 5, 6, 7, 8, 9),
  (1, 2, 3, 4, 5, 6, 7, 8, 9)])

# Análisis de Algoritmos de Búsqueda

Usaremos `CountCalls` para envolver el objeto `Problem` de tal manera que las llamadas a sus métodos se deleguen al problema original, pero cada llamada incrementa un contador. Una vez que hemos resuelto el problema, imprimimos un resumen estadísticas.

In [96]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))

In [97]:
report([astar_search, bidirectional_astar_search, uniform_cost_search, bidirectional_uniform_cost_search], [c1, c2, c3, c4])

astar_search:
    1,542 nodes |      258 goal |    7 cost |     264 actions | PancakeProblem((4, 6, 2, 5, 1, 3), (1, 2
    4,438 nodes |      635 goal |    8 cost |     642 actions | PancakeProblem((1, 3, 7, 5, 2, 6, 4), (1
      343 nodes |       50 goal |    6 cost |      55 actions | PancakeProblem((1, 7, 2, 6, 3, 5, 4), (1
    2,538 nodes |      283 goal |    9 cost |     291 actions | PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 
    8,861 nodes |    1,226 goal |   30 cost |   1,252 actions | TOTAL

bidirectional_astar_search:
      378 nodes |        0 goal |    7 cost |      70 actions | PancakeProblem((4, 6, 2, 5, 1, 3), (1, 2
      749 nodes |        0 goal |    8 cost |     115 actions | PancakeProblem((1, 3, 7, 5, 2, 6, 4), (1
       35 nodes |        0 goal |    6 cost |      11 actions | PancakeProblem((1, 7, 2, 6, 3, 5, 4), (1
      126 nodes |        0 goal |    9 cost |      23 actions | PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 
    1,288 nodes |        0 goal |   30 cost |  

Necesitamos explorar casi 300 veces más nodos sin la heurística.